# 1. request 패키지

In [ ]:
import requests

URL = 'https://www.naver.com'
response = requests.get(URL) #get 방식으로 요청
print(response.status_code)

In [ ]:
print(response.text) #네이버 홈페이지에 해당하는 html

In [ ]:
URL = 'https://search.naver.com/search.naver?query=python'
query = {'query':'python'}
response = requests.get(URL, params=query)
print(response.status_code)
print(response.text)

# 2. user-agent 값 설정
- 로봇이 아님을 나타내기 위해서 user-agent라는 값을 header에 넣어서 보냄
- 직접적인 URL 주소로 요청 시 웹 사이트에서 웹 크롤링을 통해 접근한 것을 감지하고 접속을 차단하게 된다
- user-agent 헤더값을 포함하여 요청하면 브라우저를 통해 요청하는 것으로 인식되어 해결
- 웹 브라우저 실행 -> F12 개발자 모드 진입 -> Console에 navigator.userAgent 입력


In [ ]:
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

In [5]:
import requests

URL = 'http://www.google.com/search'
params = {'q':'python'} #구글에서는 맵핑을 할 때 q라는 키워드에다 저장을 한다
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36' } #로봇이 아님을 나타내는 user-agent값을 헤더에 넣어 보낸다
response = requests.get(URL, params=params, headers=headers)
response.raise_for_status() #응답코드가 200이 아니면 오류내고 멈춰라
result = response.text

with open('mygoogle.html','w', encoding='utf-8')as f: #f라는 이름에 파일 객체를 담겠다
    f.write(result)
print('저장완료!')

저장완료!


## [실습] 네이버 데이터랩에서 실시간 인기 검색어 추출

In [16]:
#페이지에서  f12를 누르고 개발자 모드의 좌측상당의 화살표 select모드 선택 후 추출하고 싶은 단어를 클릭하면 단어의 위치를 알려준다

#추출하고 싶은 정보에 바로 접근하는 건 힘들고 추출하고자 하는 정보의 상위/형제 태그를 이용해 하위/같은 레벨의 태그에 접근한다

import requests

response = requests.get('https://datalab.naver.com') #요청을 해야하는 웹 사이트의 주소 입력
html_text = response.text #넘어온 응답 메세지를 html_text에 담는다
#근접한/같은 레벨의 em이라는 class로 split(분리)한다.
temp = html_text.split('<em class="num">1</em>')[1]
#em태그의 클래스는 num이라는 값을 가지고 있다.
temp = temp.split('<span class="title">')[1]
#바로 위에서 만들어진 템프를 다시 <span>태그로 split, 속성들이 대부분 ""로 묶여 있기 때문에 ''로 묶어준다.
temp = temp.split('</span>')[0] #다시 </span>태그로 스플릿
print(temp)

트위드자켓


# 3. BeautifulSoup 패키지

### 3.1 Parser별 출력 결과 비교

In [18]:
#!pip install html5lib
!pip install lxml

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<a></p>','html.parser')
print('html.parcer')
print(soup)
print('-'*40)

soup = BeautifulSoup('<a></p>','lxml')
print('lxml')
print(soup)
print('-'*40)

soup = BeautifulSoup('<a></p>','xml')
print('xml')
print(soup)
print('-'*40)

soup = BeautifulSoup('<a></p>','html5lib')
print('html5lib')
print(soup)
print('-'*40)

html.parcer
<a></a>
----------------------------------------
lxml
<html><body><a></a></body></html>
----------------------------------------
xml
<?xml version="1.0" encoding="utf-8"?>
<a/>
----------------------------------------
html5lib
<html><head></head><body><a><p></p></a></body></html>
----------------------------------------


## 3.2 기본 사용법

In [12]:
import requests
from bs4 import BeautifulSoup
#위키피디아 한
URL = 'https://ko.wikipedia.org/wiki/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%9F%AC' #마지막 부분은 유니코트도 표현된 것 -웹크롤러

response = requests.get(URL)

#soup 객체를 생성하기
soup = BeautifulSoup(response.text, 'lxml')

#태그를 이용한 접근
print(soup.title)
print(soup.footer.ul.li)

#태그는 지우고 안의 값만 추출하기
print(soup.footer.ul.li.text)

#태그와 속성을 이용한 접근
print(soup.a) #soup 객체에서 첫번째로 만나는 a태그를 출력

#print(soup.a['id']) #만약 속성이 존재하지 않으면 에러 발생

print(soup.a['href'])

#find 함수를 이용한 태그 내의 다양한 속성을 이용한 접근
print(soup.find('a',attrs={'title':'구글봇'})) #a태그 중 title 속성의 값이 '구글봇'인 데이터를 검색

<title>웹크롤러 - 위키백과, 우리 모두의 백과사전</title>
<li id="footer-info-lastmod"> 이 문서는 2023년 10월 12일 (목) 12:40에 마지막으로 편집되었습니다.</li>
 이 문서는 2023년 10월 12일 (목) 12:40에 마지막으로 편집되었습니다.
<a class="mw-jump-link" href="#bodyContent">본문으로 이동</a>
#bodyContent
None


## 3.3 자식 노드를 반복 가능한 객체로 반환

In [3]:
html = '''
<html>
    <head>
        <title>Web Scrapping<\title>
    <\head>
    <body>
        <p class="a" align="center">text1<\p>
        <p class="b" align="center">text2<\p>
        <p class="c" align="center">text3<\p>
        <div>
            <img src="/source" width="300" height="200">
        <\div>
    <\body>
<\html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
contens = soup.find('body')
#print(contens)

for child in contens.children:
    print(child)



<p align="center" class="a">text1&lt;\p&gt;
        <p align="center" class="b">text2&lt;\p&gt;
        <p align="center" class="c">text3&lt;\p&gt;
        <div>
<img height="200" src="/source" width="300"/>
        &lt;\div&gt;
    &ltody&gt;
&lt;\html&gt;
</div></p></p></p>


## 3.4 자신을 포함한 부모 노드까지 출력하기

In [5]:
html = '''
<html>
    <head>
        <title>Web Scrapping<\title>
    <\head>
    <body>
        <p class="a" align="center">text1<\p>
        <p class="b" align="center">text2<\p>
        <p class="c" align="center">text3<\p>
        <div>
            <img src="/source" width="300" height="200">
        <\div>
    <\body>
<\html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
contens = soup.find('body')
img_tag = contens.find('img')
print(img_tag)
print()
print(img_tag.parent)



<img height="200" src="/source" width="300"/>

<div>
<img height="200" src="/source" width="300"/>
        &lt;\div&gt;
    &ltody&gt;
&lt;\html&gt;
</div>


## 3.5 특정 부모 노드까지 검색해서 올라감

In [ ]:
html = '''
<html>
    <head>
        <title>Web Scrapping<\title>
    <\head>
    <body>
        <p class="a" align="center">text1<\p>
        <p class="b" align="center">text2<\p>
        <p class="c" align="center">text3<\p>
        <div>
            <img src="/source" width="300" height="200">
        <\div>
    <\body>
<\html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
contens = soup.find('body')
img_tag = contens.find('img')
print(img_tag.find_parent('body')) #자기 자신을 기준으로 지정된 부모 태그까지 검색해서 올라가기

## 3.6 형제 노드 검색

In [10]:
html = '''
<html>
    <head>
        <title>Web Scrapping<\title>
    <\head>
    <body>
        <p class="a" align="center">text1<\p>
        <p class="b" align="center">text2<\p>
        <p class="c" align="center">text3<\p>
        <div>
            <img src="/source" width="300" height="200">
        <\div>
    <\body>
<\html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
p_tag = soup.find('p', attrs={'class':'b'}) #클래스 속성의 값이 b를 가지고 있는 p태그를 찾아라
print(p_tag)
print('-- 바로 다음 형제 노드 --')
print(p_tag.find_next_sibling())
print('-- 모든 다음 형제 노드 --')
print(p_tag.find_next_siblings())
print('-- 바로 이전 형제 노드 --')
print(p_tag.find_previous_sibling())
print('-- 모든 이전 형제 노드 --')
print(p_tag.find_previous_siblings())

<p align="center" class="b">text2&lt;\p&gt;
        <p align="center" class="c">text3&lt;\p&gt;
        <div>
<img height="200" src="/source" width="300"/>
        &lt;\div&gt;
    &ltody&gt;
&lt;\html&gt;
</div></p></p>
-- 바로 다음 형제 노드 --
None
-- 모든 다음 형제 노드 --
[]
-- 바로 이전 형제 노드 --
None
-- 모든 이전 형제 노드 --
[]


## 3.7 검색 : find()

In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://www.naver.com')
soup = BeautifulSoup(response.text, 'lxml')
print(soup.find('title'))
print(soup.find('a'))
print(soup.find(id='search')) #id속성의 값이 search인 정보를 가져옴, soup.find(attrs={'id:'search'})와 동일

## 3.8검색 : find_all()

In [19]:
a_tags = soup.find_all('a')
print(len(a_tags))
print(a_tags)
#a_tags = soup.find_all('a', limit = 2) 가장 처음 만나는 두개의 a 태그

27
[<a href="#topAsideButton"><span>상단영역 바로가기</span></a>, <a href="#shortcutArea"><span>서비스 메뉴 바로가기</span></a>, <a href="#newsstand"><span>새소식 블록 바로가기</span></a>, <a href="#shopping"><span>쇼핑 블록 바로가기</span></a>, <a href="#feed"><span>관심사 블록 바로가기</span></a>, <a href="#account"><span>MY 영역 바로가기</span></a>, <a href="#widgetboard"><span>위젯 보드 바로가기</span></a>, <a href="#viewSetting"><span>보기 설정 바로가기</span></a>, <a aria-pressed="false" class="item _delAll" href="#" role="button">전체삭제</a>, <a class="kwd_help" data-clk="sly.help" href="https://help.naver.com/alias/search/word/word_35.naver" target="_blank">도움말</a>, <a class="kwd_help" data-clk="sly.help" href="https://help.naver.com/alias/search/word/word_35.naver" target="_blank">도움말</a>, <a class="close _keywordOnOff" href="#">자동저장 끄기</a>, <a data-clk="sly.help" href="https://help.naver.com/alias/search/word/word_35.naver" target="_blank">도움말</a>, <a class="close _close" href="#">닫기</a>, <a aria-pressed="false" class="btn_help _tg_btn" href=

[실습] 네이버 뉴스 페이지에서 언론사 목록 가져오기

In [6]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://news.naver.com')
soup = BeautifulSoup(res.text, 'lxml')
result = soup.find_all('h4', attrs={'class':'channel'})
#print(len(result))
print(result[0]) #태그를 없애고 텍스트 값만 읽어오고 싶으면 print(result[0].get_text()) 또는 print(result[0].text)이용

print(list(result[0].children))
#['부산일보', <span class="datetime">04월 05일 09:52</span>]

press_list = [ list(tag.children)[0] for tag in result]
print(press_list[:10])


<h4 class="channel">서울경제<span class="datetime">04월 05일 10:19</span></h4>
['서울경제', <span class="datetime">04월 05일 10:19</span>]
['서울경제', '주간조선', '경향신문', '뉴시스', '대전일보', '전주MBC', '채널A', '코리아중앙데일리', 'MBC', '헬스조선']


In [7]:
res = requests.get('https://news.naver.com')
soup = BeautifulSoup(res.text, 'lxml')
result = soup.find_all('div', attrs={'class':'cjs_age_name'})
print(press_list[:10])

['서울경제', '주간조선', '경향신문', '뉴시스', '대전일보', '전주MBC', '채널A', '코리아중앙데일리', 'MBC', '헬스조선']


## 3.9 검색 : select_one(), select()

In [11]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://www.tradecampus.com')
soup = BeautifulSoup(res.text, 'html.parser')

print(soup.select_one('div > a'))
result = soup.select('div a')
print(len(result))

<a class="show" href="/page/KITA_MAIN">KITA 무역아카데미</a>
220


In [13]:
print(soup.select_one('body > div > div.wrapper.main_page > div.renew_main > div.col-12 > div > div.renew_main_notice > div > div > h3'))

#텍스트 값만 읽어오고 싶을때
print(soup.select_one('body > div > div.wrapper.main_page > div.renew_main > div.col-12 > div > div.renew_main_notice > div > div > h3').text)


<h3 class="tit">공지사항</h3>
공지사항


In [20]:
#tradecampus.com 메인 페이지 공지사항 두번째 항목 선택
notice = soup.select('body > div > div.wrapper.main_page > div.renew_main > div.col-12 > div > div.renew_main_notice > div > ul > li:nth-child(2) > p > a') 
print(notice)

#:nth-child(#)을 지우면 해당 요소(element)의 모든 요소를 가져온다
notice = soup.select('body > div > div.wrapper.main_page > div.renew_main > div.col-12 > div > div.renew_main_notice > div > ul > li > p > a')
print(notice) #전체목록 가져오기

notice = soup.select('body > div > div.wrapper.main_page > div.renew_main > div.col-12 > div > div.renew_main_notice > div > ul > li > p > a')
print(notice[0].text)


[<a href="/page/user_academy_service_notice?post_seq=20424">제31기 수출입기업 영업이익의 보전을 위한 원스톱 환리스크관리 개강(3/21)</a>]
[<a href="/page/user_academy_service_notice?post_seq=20506">2024년 무역아카데미 주요강좌 안내</a>, <a href="/page/user_academy_service_notice?post_seq=20424">제31기 수출입기업 영업이익의 보전을 위한 원스톱 환리스크관리 개강(3/21)</a>, <a href="/page/user_academy_service_notice?post_seq=20413">관세사의 FTA 활용꿀팁 전수! e러닝 안내(원산지관리전담자 교육점수 인정)</a>]
2024년 무역아카데미 주요강좌 안내


## 3.10 텍스트 가져오기 : text, get_tesxt()
-  검색 결과에서 태그를 제외한 텍스트만 추출

In [23]:
#고객 센터 영역 텍스트 가져오기
tag = soup.find('div', attrs={'class':'serviceInfo'})
#print(tag)
print(tag.get_text())


고객센터


오프라인 교육, e러닝
02-6000-5378/5379




운영시간
평일 09:00~18:00 (주말/공휴일 : 휴무)





In [25]:
tag = soup.find('img', attrs = {'class':'mobile_icon black'})
print(tag['src'])
print(tag.get('src'))

/weven_template_repository/theme/KITAAC/1/resource/img/ico_sns_facebook.png
/weven_template_repository/theme/KITAAC/1/resource/img/ico_sns_facebook.png


## 3.11 텍스트 가져오기:string
- 검색 결과에서 **태그 안에 또 다른 태그가 없는 경우** 해당 내용을 출력

In [28]:
tag = soup.find('div', attrs={'class':'serviceInfo'})
tag = tag.find('span')
print(tag.string)

오프라인 교육, e러닝


[실습] 네이버 웹툰 제목 가져오기

In [ ]:
!pip install selenium

In [34]:
!pip install webdriver_manager

In [31]:
import requests
from bs4 import BeautifulSoup

URL = 'https://comic.naver.com/webtoon'
res = requests.get(URL)
soup = BeautifulSoup(res.text, 'lxml')

#아래 두가지 방법으로는 찾을 수가 없다 > selenium 이용
#webtoon_titles = soup.find_all('span',attrs = {'class':'ContentTitle__title--e3qXt'})
#print(len(webtoon_titles))

#webtoon_titles = soup.select_one('#container > div.component_wrap.type2 > div.WeekdayMainView__daily_all_wrap--UvRFc > div:nth-child(1) > ul > li:nth-child(1) > div > a > span > span')
#print(webtoon_titles)




None


In [11]:
import requests
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
# 위에는 패키지를 동적으로 다운 받아 실행하는 방법

URL = 'https://comic.naver.com/webtoon'
driver.get(URL)
time.sleep(4) # 동적으로 생성되는 페이지의 내용이 완성될 때까지 대기
soup = BeautifulSoup(driver.page_source, 'lxml')

# 요일별 전체 웹툰 CSS 선택자
temp = soup.select_one('#container > div.component_wrap.type2 > div.WeekdayMainView__daily_all_wrap--UvRFc') # '#container' -> '#' = 아이디선택자
#print(temp)

#요일별 div 태그 검색
temp = temp.find_all('div', attrs={'class':'WeekdayMainView__daily_all_item--DnTAH'})
print(len(temp))

week = ['월', '화', '수','목', '금', '토','일']
for i, w in enumerate(temp):
    print(f'==== {week[i]}요 웹툰====')
    webtoon_list = w.find_all('li', attrs={'class':'DailyListItem__item--LP6_T'})
    for webtoon in webtoon_list:
        print(webtoon.find('span', attrs={'class':'text'}).text)
    print()
    

7
==== 월요 웹툰====
뷰티풀 군바리
참교육
백수세끼
신의 탑
왕과의 야행
장씨세가 호위무사
퀘스트지상주의
윈드브레이커
소녀의 세계
신화급 귀속 아이템을 손에 넣었다
버림받은 왕녀의 은밀한 침실
팔이피플
절대검감
별난식당
귀촌리
환생천마
회귀한 공작가의 막내도련님은 암살자
우아한 욕망
어린이집 다니는 구나
만렙돌파
진주
수상한 다이어트 클럽
여고생이 신인데 나만 괴롭힘
마왕을 그만둔 이유
북부 공작님을 유혹하겠습니다
개같이 탈출
악당 가족이 독립을 반대한다
녹빛자정의 연인
인섹터
히어로메이커
제왕
더블클릭
토마토가 돼라!
리턴 투 플레이어
칼가는 소녀
영업 천재가 되었다
좀비묵시록 82-08
미친 재능의 플레이어
막장 악녀
결혼생활 그림일기
루루라라 우리네 인생
아슈타르테
찔레꽃 그늘 아래
파운더
함부로 친절하지 말라
보스리턴
오빠집이 비어서
사랑, 그거 어떻게 하는 건데
헬크래프트
시한부의 아이까지 뺏으려 합니다
꿈의 기업
공녀님의 꽃밭에는 그들이 산다
내가 왜 킬러?!
엘프
악녀교실
제국 제일의 상속녀가 되었습니다
K학원 생존기
쥴리에타의 드레스 업
영앤리치가 아니야!
반려짐승
달로 만든 아이
사이다걸
두 번째 삶은 힐링라이프?
집사, 주세요!
백호랑
찌질하지만 로맨스는 하고 싶어
마이너스의 손
학식의 꿈
프로페서
다육이는 잘 자란다
최후의 금빛아이
고백어택
어쌔신 크리드 - 잊혀진 사원
컨트롤X
날 먹는 건 금지양!
나의 보이소프렌드
역주행!
포스트 팬데믹
선빵필승!
주작연애
티엔다비스 - 완벽한 구원을 위하여
처음을 줄게!
오늘의 일기예보
버그이터
러브 똘츄얼리
트리거
악취해결사
어느날 짝남에게 공작님이 빙의했다

==== 화요 웹툰====
좋아? 죽어!
김부장
마음의소리2
서울 자가에 대기업 다니는 김 부장 이야기
내가 키운 S급들
마루는 강쥐
괴력 난신
유부 감자
사신소년
멸망 이후의 세계
한림체육관
천마는 평범하게 살 수 없다
하루만 네가 되고 싶어
욕망일기Deep
집이 없어
이섭의 연애
궤짝
유사연애
신입사원 김철수
성검전설
하북팽가 막내

[실습] 메가박스 영화정보 사이트에서 영화 포스터 다운로드 하기

In [18]:
# 사전 테스트 : 박스오피스 1위 영화 포스터 이미지 가져오기

from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriver

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())

URL = 'https://www.megabox.co.kr/movie'
driver.get(URL)

soup = BeautifulSoup(driver.page_source, 'lxml')
poster_img = soup.select('#movieList > li > div.movie-list-info > img')
print(len(poster_img))
poster_img_src = poster_img[0].get('src')

import requests
res = requests.get(poster_img_src)
with open('poster.jpg', 'wb') as f:
    f.write(res.content)
print('End')

20
End


In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
URL = 'https://www.cgv.co.kr/movies'
driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'lxml')
poster_img = soup.select('#contents > div.wrap-movie-chart > div.sect-movie-chart > ol:nth-child(2) > li:nth-child(1) > div.box-image > a > span > img')

[문제] 메가박스 영화 사이트에서 첫 페이지에 있는 모든 영화 포스트 이미지 수집하기
- 메가박스 영화 사이트 첫 페이지에 있는 20개의 영화 포스트 이미지 수집
- 현재 작업 디렉토리 밑에 'poster_img'폴더가 없는 경우 폴더를 생성한다.(os 패키지 디렉토리 이용)
- 저장되는 각 포스터 이미지의 파일이름은 영화 제목으로 한다.

In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
import requests

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
URL = 'https://www.megabox.co.kr/movie'
driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'lxml')

# 포스터 이미지를 가지고 있는 모든 img 태그를 검색
poster_imgs = soup.find_all('img', attrs={'class':'poster lozad'})
# print(len(poster_imgs))

# 이미지를 저장할 폴더 생성
import os
img_dir = './poster_img/' # .은 현재 경로
if not os.path.exists(img_dir):
    os.mkdir(img_dir)
    print('폴더 생성 완료')
else:
    print('폴더가 존재함')

for i, poster in enumerate(poster_img, 1): # 시작값 1 입력함. 디폴트 0
    title = poster.get('alt') # 영화 제목 속성 poster[alt]도 가능
    img_url = poster.get('src') # 이미지 파일 poster[src]도 가능

    print(i, ':', img_url)
    img_res = requests.get(img_url)

    if ':' in title:
        title = title.replace(':',' ')

    with open(img_dir+f'[{i}].(title).jpg', 'wb') as f:
        f.write(img_res.content)

폴더 생성 완료
1 : https://img.cgv.co.kr/Movie/Thumbnail/Poster/000088/88077/88077_320.jpg


[실습] 네이버 뉴스 사이트에서 경제 관련 언론사별 랭킹뉴스 추출하기
- 언론사 이름에 '경제'단어가 포함된 언론사의 랭킹 뉴스만 추출하여 출력한다.

In [2]:
import requests
from bs4 import BeautifulSoup

URL = 'https://news.naver.com/main/ranking/popularDay.naver'
res = requests.get(URL)
soup = BeautifulSoup(res.text, 'html.parser')

news_list = soup.find_all('div',attrs={'class':'rankingnews_box'})
print('등록 언론사 개수:', len(news_list))
for news in news_list:
    press_title = news.find('strong').text
    if '경제' in press_title:
        print('언론사:',press_title)
        press_news = news.find_all('div',attrs={'class':'list_content'})
        for i, ranking_news in enumerate(press_news, 1):
            print(f'{i}:{ranking_news.find("a").get_text()}')
        print()


등록 언론사 개수: 83
언론사: 아시아경제
1:"땅 팔아요, 단 중국인은 사지 마세요"… '노 차이나' 확산하는 美
2:"만성피로는 소리없는 살인자"…걸그룹 멤버도 무기한 활동중단
3:"불교가 이렇게 힙할 줄 몰랐다"…불교박람회 때아닌 흥행몰이
4:경치 좋은 곳에서 '숲멍'하고 돈 받는 직업…"연봉 5천 이상"
5:尹, '이재명 헬기 이송 논란' 부산대병원 권역외상센터 방문…"전폭적 지원"

언론사: 매일경제
1:“손흥민 어쩌나”...토트넘 구단주, 불법 주식거래 미국서 ‘유죄’ 확정
2:“가급적 나가지 마세요”…한달치 비 한꺼번에 쏟아진 시드니, 항공편 100여편 취소
3:“여성신체 과도하게 드러내더니”…수원 이어 파주서도 무산된 성인페스티벌
4:지진 몰아치는데…식당 문열고 택시·기차 정상운영한 비결은 [대만 지진르포]
5:명룡대전 무게추 옮긴 3040 … 이재명 53%·원희룡 40%

언론사: 서울경제
1:항공편 100편 이상 취소에 댐 범람 위험…하루에 한달치 비 퍼부은 '이 나라'
2:"술 취해서 자는 거예요" 홀로 떠난 男…여관방서 50대女 숨진 채 발견
3:58세 할머니 레깅스 입고 손주들 앞서 '이 자세'로 4시간 반 버티다 결국…
4:한동훈 "높은 사전투표율, 우리가 뭉친다는 얘기"
5:기초 수급자로…평생 아껴 모은 전 재산 기부하고 홀로 떠난 할머니 사연에 '눈물이 왈칵'

언론사: 한국경제
1:대놓고 '조국당' 힘실은 文…"탈당하라" 이재명 지지자 분노
2:"수입 얼마나 줄었길래"…전공의 떠난 대형병원 '파산 위기'
3:'성시경 막걸리' 결국 일냈다…주류 전문가들 '감탄'
4:암표 잡으려다 아이유 팬 '눈물'…500만원까지 뛴 티켓 어쩌나 [연계소문]
5:1억원대 '남편 빚투'…배우 최정원 "모르는 내용, 별거 상태"

언론사: 헤럴드경제
1:“짜장면 한그릇에 배달비 4000원, 이젠 없다” 화들짝 놀란 요기요 눈물의 결단
2:“호빵 두 개 붙인 줄 알았다” 조롱받던 사진 속 헤드폰 ‘반전’…새 제품 등장
3:[단독] “아! 알리 꼼수에 당

# 4. Selenium 패키지

## 4.1 find_element() 함수

In [4]:
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

#웹 드라이버를 동적으로 다운로드해서 실행abs
driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
#[참고] 로컬 컴퓨터에 설치되어 있는 웹 드라이버 실행 방식
#s = Service('d:\DEV\chromedriver\chromedriver.exe') #경로지정 예시
#driver = Chrome(service=s)

driver.get('https://www.daum.net')
ele = driver.find_element(by=By.LINK_TEXT,value='카페')
print(type(ele))
print(ele.text)


<class 'selenium.webdriver.remote.webelement.WebElement'>
카페


## 4.2 이벤트 제어하기

### 4.2.1 click()

In [7]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())

driver.get('https://www.naver.com')
ele = driver.find_element(by=By.CSS_SELECTOR, value='#shortcutArea > ul > li:nth-child(5) > a')
ele.click()


### 4.2.2 send_keys()

In [9]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys #특수키 사용을 위한 클래스

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())

driver.get('https://www.naver.com')
ele = driver.find_element(by=By.ID, value='query')
ele.send_keys('python')
ele.send_keys(Keys.ENTER)

[실습] 네이버 로그인 하기

In [16]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())

driver.get('https://www.naver.com')
ele = driver.find_element(by=By.CLASS_NAME, value='MyView-module__link_login___HpHMW')
ele.click()

my_id = 'dnrkdus1'
my_pw = 'Wyeon3936'

#로봇에 의해 클릭되지 못하도록 막았기 때문에 스크립트로 처리해야함abs
#ele = driver.find_element(by=By.ID, value='id')
#ele.send_keys(my_id)

#ele = driver.find_element(by=By.ID, value='pw')
#ele.send_keys(my_pw)

driver.execute_script(f"document.getElementById('id').value = '{my_id}'")
driver.execute_script(f"document.getElementById('pw').value = '{my_pw}'")

ele = driver.find_element(by=By.CLASS_NAME, value ='btn_login')
ele.click()

In [19]:
#로그인 상태 유지하기

from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())

driver.get('https://www.naver.com')
ele = driver.find_element(by=By.CLASS_NAME, value='MyView-module__link_login___HpHMW')
ele.click()

my_id = 'dnrkdus1'
my_pw = 'Wyeon3936'

#로봇에 의해 클릭되지 못하도록 막았기 때문에 스크립트로 처리해야함abs
#ele = driver.find_element(by=By.ID, value='id')
#ele.send_keys(my_id)

#ele = driver.find_element(by=By.ID, value='pw')
#ele.send_keys(my_pw)

driver.execute_script(f"document.getElementById('id').value = '{my_id}'")
driver.execute_script(f"document.getElementById('pw').value = '{my_pw}'")

#로그인 상태 유지 체크박스 클릭
driver.execute_script(f'document.getElementById("keep").value = "on"')

ele = driver.find_element(by=By.CLASS_NAME, value ='btn_login')
ele.click()

## 4.4 웹브라우저 자동 스크롤

### 4.4.1 구글에서 이미지 검색 후 검색결과 6번 스크롤 하기

In [25]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

#페이지가 로드될때까지 기다리는 시간
SCROLL_PAUSE_TIME = 2

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())
driver.get('https://www.google.com')
ele = driver.find_element(by=By.CLASS_NAME, value='gLFyf')
ele.send_keys('python')
ele.submit()

driver.find_element(By.LINK_TEXT, '이미지').click()

#페이지가 로드될때까지 기다림
time.sleep(SCROLL_PAUSE_TIME)
#최초 스크롤바의 높이값 읽기
last_height = driver.execute_script('return document.body.scrollHight')
print('last height:', last_height)
for i in range(6):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    #(0. document.body.scrollHeight) 현재 윈도우의 스크롤 바를 0에부터 가장 밑(scrollHeight)까지 이동시켜라
    time.sleep(SCROLL_PAUSE_TIME)
    #로딩 된 후에는 새로운 높이값을 갖게 된다
    #스크롤 바 이동으로 새로운 검색 결과가 로딩 후 변경된 새로운 스크롤 바의 높이 값 읽기
    new_height = driver.execute_script('return document.body.scrollHeight')
    print('new height:', new_height)
    print('-'*30)

#더이상 스크롤 될 페이지가 없을 경우 scrollHeight 값의 변화가 없다.

last height: None
new height: 13935
------------------------------
new height: 20631
------------------------------
new height: 27079
------------------------------
new height: 27079
------------------------------
new height: 27079
------------------------------
new height: 27079
------------------------------


### 4.4.2 구글에서 이미지 검색 후 검색 결과 무한 스크롤하기

In [26]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

#페이지가 로드될때까지 기다리는 시간
SCROLL_PAUSE_TIME = 2

driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())
driver.get('https://www.google.com')
ele = driver.find_element(by=By.CLASS_NAME, value='gLFyf')
ele.send_keys('python')
ele.submit()

driver.find_element(By.LINK_TEXT, '이미지').click()

#페이지가 로드될때까지 기다림
time.sleep(SCROLL_PAUSE_TIME)
#최초 스크롤바의 높이값 읽기
last_height = driver.execute_script('return document.body.scrollHight')
print('last height:', last_height)

#스크롤 횟수
scroll_cnt = 0


while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    #(0. document.body.scrollHeight) 현재 윈도우의 스크롤 바를 0에부터 가장 밑(scrollHeight)까지 이동시켜라
    scroll_cnt += 1
    time.sleep(SCROLL_PAUSE_TIME)
    #로딩 된 후에는 새로운 높이값을 갖게 된다
    #스크롤 바 이동으로 새로운 검색 결과가 로딩 후 변경된 새로운 스크롤 바의 높이 값 읽기
    new_height = driver.execute_script('return document.body.scrollHeight')
    print(f'last height:{last_height}, new height:{new_height}, scroll count: {scroll_cnt}')
    print('-'*30)
    if last_height != new_height: # 계속해서 new_height 값이 변경되면
        last_height = new_height
    else: #더이상 nnew_height의 변경이 없으면
        break

last height: None
last height:None, new height:13915, scroll count: 1
------------------------------
last height:13915, new height:20611, scroll count: 2
------------------------------
last height:20611, new height:27059, scroll count: 3
------------------------------
last height:27059, new height:29905, scroll count: 4
------------------------------
last height:29905, new height:29905, scroll count: 5
------------------------------


### 4.4.3 구글에서 이미지 검색 후 썸네일 이미지 클릭하고 원본 이미지 주소 수집하기

In [30]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

#페이지가 로드될때까지 기다리는 시간
SCROLL_PAUSE_TIME = 2
IMAGE_EXTRACT_NUM = 20 #이미지 추출 개수
SEARCH_KEYWORD = 'python'

##### 웹 드라이버 실행 및 구글 접속 #####
driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())
driver.get('https://www.google.com')


###### 검색 #######
ele = driver.find_element(by=By.CLASS_NAME, value='gLFyf')
ele.send_keys('python')
ele.submit()

###### 이미지 검색 결과 페이지 이동 ########
driver.find_element(By.LINK_TEXT, '이미지').click()


time.sleep(SCROLL_PAUSE_TIME) #페이지가 로드될때까지 기다림


######### 이미지 검색결과 페이지 스크롤 #########
#최초 스크롤바의 높이값 읽기
last_height = driver.execute_script('return document.body.scrollHight')
print('last height:', last_height)

#스크롤 횟수
scroll_cnt = 0


while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    #(0. document.body.scrollHeight) 현재 윈도우의 스크롤 바를 0에부터 가장 밑(scrollHeight)까지 이동시켜라
    scroll_cnt += 1
    time.sleep(SCROLL_PAUSE_TIME)
    #로딩 된 후에는 새로운 높이값을 갖게 된다
    #스크롤 바 이동으로 새로운 검색 결과가 로딩 후 변경된 새로운 스크롤 바의 높이 값 읽기
    new_height = driver.execute_script('return document.body.scrollHeight')
    print(f'last height:{last_height}, new height:{new_height}, scroll count: {scroll_cnt}')
    print('-'*30)
    if last_height != new_height: # 계속해서 new_height 값이 변경되면
        last_height = new_height
    else: #더이상 nnew_height의 변경이 없으면
        break



####### 이미지 선택 및 해당 이미지 src 추출 #########
imgs = driver.find_elements(By.CSS_SELECTOR, '#rso > div > div > div.wH6SXe.u32vCb > div > div > div > div.czzyk.XOEbc > h3 > a')
print(len(imgs))
img_src_list=[]
img_cnt = 0
for img in imgs:
    try:
        img.click()
        time.sleep(1.5)
        img_src = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div.A8mJGd.NDuZHe.OGftbe-N7Eqid-H9tDt > div.LrPjRb > div.AQyBn > div.tvh9oe.BIB1wf > c-wiz > div > div > div > div > div.v6bUne > div.p7sI2.PUxBg > a > img.sFlh5c.pT0Scc.iPVvYb').get_attribute('src')
        img_cnt += 1
        print(f'[{img_cnt}]:{img_src}')
        img_src_list.append(img_src)
        if img_cnt == IMAGE_EXTRACT_NUM: break
    except:
        img_cnt -= 1 #에러가 발생했을 시 지속되도록


last height: None
last height:None, new height:7239, scroll count: 1
------------------------------
last height:7239, new height:7239, scroll count: 2
------------------------------
100
[1]:https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/800px-Python-logo-notext.svg.png
[2]:https://velog.velcdn.com/images/deep-of-machine/post/3f778fa2-2b43-42b3-9233-091424be7d73/image.png
[3]:https://i.namu.wiki/i/mxMv5lNX8m8lUwu7yTjN6eNZh8JVuI6a_chEyMRc4V9oECkhVIl7OiPiGIOllv14uDVNuwRPVco8abCPe5xOiQ.svg
[4]:https://images.velog.io/images/pm1100tm/post/30e9dc94-96d0-41b3-9f2f-d814e839a796/python.jpg
[5]:https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Python.svg/800px-Python.svg.png
[6]:https://i.namu.wiki/i/pHxeJONxIv51qQsN2ac5nX3shPEmiSlKtGVATZXUE22NHGyw9v7_Aqto6aSoCU9ODz3RKtTKCEP0E0OI7TlxMQ.webp
[7]:https://fineproxy.org/wp-content/uploads/2023/05/Python.jpg
[8]:https://www.askedtech.com/api/kords/admin/product/image.jpg?type=org&id=20688
[9]:https://store-im

### 4.4.4 구글에서 이미지 검색 후 파일로 저장하기

In [34]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

#페이지가 로드될때까지 기다리는 시간
SCROLL_PAUSE_TIME = 2
IMAGE_EXTRACT_NUM = 20 #이미지 추출 개수
SEARCH_KEYWORD = 'python'

##### 웹 드라이버 실행 및 구글 접속 #####
driver = Chrome(service=Service(ChromeDriverManager().install()),options=ChromeOptions())
driver.get('https://www.google.com')


###### 검색 #######
ele = driver.find_element(by=By.CLASS_NAME, value='gLFyf')
ele.send_keys('python')
ele.submit()

###### 이미지 검색 결과 페이지 이동 ########
driver.find_element(By.LINK_TEXT, '이미지').click()


time.sleep(SCROLL_PAUSE_TIME) #페이지가 로드될때까지 기다림


######### 이미지 검색결과 페이지 스크롤 #########
#최초 스크롤바의 높이값 읽기
last_height = driver.execute_script('return document.body.scrollHight')
print('last height:', last_height)

#스크롤 횟수
scroll_cnt = 0


while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    #(0. document.body.scrollHeight) 현재 윈도우의 스크롤 바를 0에부터 가장 밑(scrollHeight)까지 이동시켜라
    scroll_cnt += 1
    time.sleep(SCROLL_PAUSE_TIME)
    #로딩 된 후에는 새로운 높이값을 갖게 된다
    #스크롤 바 이동으로 새로운 검색 결과가 로딩 후 변경된 새로운 스크롤 바의 높이 값 읽기
    new_height = driver.execute_script('return document.body.scrollHeight')
    print(f'last height:{last_height}, new height:{new_height}, scroll count: {scroll_cnt}')
    print('-'*30)
    if last_height != new_height: # 계속해서 new_height 값이 변경되면
        last_height = new_height
    else: #더이상 nnew_height의 변경이 없으면
        break



####### 이미지 선택 및 해당 이미지 src 추출 #########
imgs = driver.find_elements(By.CSS_SELECTOR, '#rso > div > div > div.wH6SXe.u32vCb > div > div > div > div.czzyk.XOEbc > h3 > a')
print(len(imgs))
img_src_list=[]
img_cnt = 0
for img in imgs:
    try:
        img.click()
        time.sleep(1.5)
        img_src = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div.A8mJGd.NDuZHe.OGftbe-N7Eqid-H9tDt > div.LrPjRb > div.AQyBn > div.tvh9oe.BIB1wf > c-wiz > div > div > div > div > div.v6bUne > div.p7sI2.PUxBg > a > img.sFlh5c.pT0Scc.iPVvYb').get_attribute('src')
        img_cnt += 1
        print(f'[{img_cnt}]:{img_src}')
        img_src_list.append(img_src)
        if img_cnt == IMAGE_EXTRACT_NUM: break
    except:
        img_cnt -= 1 #에러가 발생했을 시 지속되도록

###### 검색 이미지 파일로 저장 ########
import os
import requests

ts = time.localtime()
path = 'c:/Temp/'
now = '{}.{}.{}.{}.{}.{}'.format(ts.tm_year, ts.tm_mon, ts.tm_mday, ts.tm_hour, ts.tm_min, ts.tm_sec)
dir = SEARCH_KEYWORD+'/'+now+'/'
os.chdir(path)
if not os.path.exists(dir):
    os.makedirs(dir)

file_no = 1
os.chdir(path+dir)
for url in img_src_list:
    extension = url.split('.')[-1] #원본 이미지에서 가져온 확장자
    ext = ''
    if extension in ['jpg','JPG','jpeg','JPEG','png','PNG','gif','GIF']:
        ext = '.'+extension
    else:
        ext = '.jpg'

    file_name = str(file_no)+'-'+SEARCH_KEYWORD+ext
    file_no += 1
    res = requests.get(url)
    with open(file_name, 'wb') as f:
        f.write(res.content)
driver.close()

last height: None
last height:None, new height:13935, scroll count: 1
------------------------------
last height:13935, new height:20631, scroll count: 2
------------------------------
last height:20631, new height:27214, scroll count: 3
------------------------------
last height:27214, new height:30193, scroll count: 4
------------------------------
last height:30193, new height:30193, scroll count: 5
------------------------------
440
[1]:https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/800px-Python-logo-notext.svg.png
[2]:https://velog.velcdn.com/images/deep-of-machine/post/3f778fa2-2b43-42b3-9233-091424be7d73/image.png
[3]:https://i.namu.wiki/i/mxMv5lNX8m8lUwu7yTjN6eNZh8JVuI6a_chEyMRc4V9oECkhVIl7OiPiGIOllv14uDVNuwRPVco8abCPe5xOiQ.svg
[4]:https://images.velog.io/images/pm1100tm/post/30e9dc94-96d0-41b3-9f2f-d814e839a796/python.jpg
[5]:https://i.namu.wiki/i/pHxeJONxIv51qQsN2ac5nX3shPEmiSlKtGVATZXUE22NHGyw9v7_Aqto6aSoCU9ODz3RKtTKCEP0E0OI7TlxMQ.webp
[6]:h

In [ ]:
URL = 'https://www.selenium.dev/selenium/web/formPage.html'

In [7]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select # select form 제어 클래스 import함.`

URL = 'https://www.selenium.dev/selenium/web/formPage.html'

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
driver.get(URL)

ele = driver.find_element(By.NAME, 'selectomatic')
select = Select(ele)

#인덱스 기준으로 선택하기
#select.select_by_index(2)

#보여지는 선택값 텍스트로 선택하기
#select.select_by_visible_text('Four')

#option 요소의 값으로 선택하기
#select.select_by_value('four')

#select 태그에 onchange 속성이 있어서 select 클래스를 사용할 수 없을 때
driver.find_element(By.CSS_SELECTOR,'option[value="four"]').click()

[실습] yes24에서 파이썬 도서 검색하기
- yes24 사이트에서 파이썬 도서 검색 -> 검색 결과를 120개 선택
- 검색 결과에서 도서 평점이 9.6 이상인 도서 제목과 가격, 평점 출력한다.

In [9]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select # select form 제어 클래스 import함.`

URL = 'https://www.yes24.com'

driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
driver.get(URL)

ele=driver.find_element(By.ID, 'query')
ele.send_keys('파이썬')
ele.send_keys(Keys.ENTER)

driver.find_element(By.CSS_SELECTOR, '#stat_gb > option[value="120"]').click()

In [32]:
# 데이터 가져오기
import requests
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'lxml')
book_list = soup.find('ul', attrs={'id':'yesSchList'})
books = book_list.find_all('li', attrs={'':''})
print('검색 도서 권수:', len(books))

count=0
for i, book in enumerate(books, 1): # index(i) 시작값 1로 설정. 'li'에 다른 정보 태그들이 있으므로 index가 무의미해짐. 따라서 enumerate 사용 안 해도 무방. 예컨대 book in books도 가능
    rating = book.find('span', attrs={'class':'rating_grade'})
    if not rating: continue # 평점이 없는 책은 None -> False 리턴. 이 경우 이하 내용 실행을 skip함. 다시 윗줄로.
    rating = float(rating.find('em').get_text()) # soup find 함수 -> string값 리턴
    if rating > 9.6:
        count+=1
        title = book.find('a', attrs={'class':'gd_name'}).get_text()
        price = book.find('strong', attrs={'class':'txt_num'}).get_text()
        print(f'{count:03d} | {title} | {price} | 평점: {rating}')
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.106)
Stacktrace:
	GetHandleVerifier [0x00BB4CE3+225091]
	(No symbol) [0x00AE4E31]
	(No symbol) [0x00989A7A]
	(No symbol) [0x0096E312]
	(No symbol) [0x009E517B]
	(No symbol) [0x009F55A6]
	(No symbol) [0x009DF2F6]
	(No symbol) [0x009B79B9]
	(No symbol) [0x009B879D]
	sqlite3_dbdata_init [0x01029A83+4064547]
	sqlite3_dbdata_init [0x0103108A+4094762]
	sqlite3_dbdata_init [0x0102B988+4072488]
	sqlite3_dbdata_init [0x00D2C9E9+930953]
	(No symbol) [0x00AF0804]
	(No symbol) [0x00AEAD28]
	(No symbol) [0x00AEAE51]
	(No symbol) [0x00ADCAC0]
	BaseThreadInitThunk [0x75497BA9+25]
	RtlInitializeExceptionChain [0x7787BDAB+107]
	RtlClearBits [0x7787BD2F+191]


[실습] 메가박스 영화 감상평

In [39]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

PAUSE_TIME = 2.5

URL = 'https://www.megabox.co.kr/movie'
driver = Chrome(service=Service(ChromeDriverManager().install()), options=ChromeOptions())
driver.get(URL)
movie = driver.find_element(By.CSS_SELECTOR, '#movieList > li:nth-child(4) > div.movie-list-info > div.movie-score > a').send_keys(Keys.ENTER) # 파묘 클릭. click()함수 에러 발생 시 ENTER로 대체 가능.
time.sleep(PAUSE_TIME)
driver.find_element(By.LINK_TEXT, '실관람평').click()

In [40]:
ratings = []
comments = []

bs = BeautifulSoup(driver.page_source, 'lxml')
result = bs.find_all('li', attrs = {'class':'type01 oneContentTag'})
for c in result:
    rating = int(c.find('div', attrs = {'class':'story-point'}).text)
    comment = c.find('div', attrs ={'class':'story-txt'}).text.strip()
    print(rating, comment)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.106)
Stacktrace:
	GetHandleVerifier [0x00BB4CE3+225091]
	(No symbol) [0x00AE4E31]
	(No symbol) [0x00989A7A]
	(No symbol) [0x0096E312]
	(No symbol) [0x009E517B]
	(No symbol) [0x009F55A6]
	(No symbol) [0x009DF2F6]
	(No symbol) [0x009B79B9]
	(No symbol) [0x009B879D]
	sqlite3_dbdata_init [0x01029A83+4064547]
	sqlite3_dbdata_init [0x0103108A+4094762]
	sqlite3_dbdata_init [0x0102B988+4072488]
	sqlite3_dbdata_init [0x00D2C9E9+930953]
	(No symbol) [0x00AF0804]
	(No symbol) [0x00AEAD28]
	(No symbol) [0x00AEAE51]
	(No symbol) [0x00ADCAC0]
	BaseThreadInitThunk [0x75497BA9+25]
	RtlInitializeExceptionChain [0x7787BDAB+107]
	RtlClearBits [0x7787BD2F+191]


In [41]:
# 페이지 번호 클릭 테스트
driver.find_element(By.CSS_SELECTOR, '#contentData > div > div.movie-idv-story > nav > a.control.last').click() # 첫 페이지 CSS 선택자
time.sleep(PAUSE_TIME)
total_page_num = int(driver.find_element(By.CSS_SELECTOR, '#contentData > div > div.movie-idv-story > nav > strong').text) # 마지막 페이지 CSS 선택자 -> for문 반복회수 확인

print('전체 페이지 수:', total_page_num)
print('-'*80)
# 다시 첫 페이지로 이동
driver.find_element(By.CSS_SELECTOR, '#contentData > div > div.movie-idv-story > nav > a.control.first').click()
time.sleep(PAUSE_TIME)

ratings = [] # 평점 정보 저장할 리스트
comments = [] # 관람평

THE_LAST_PAGE = 20 #마지막 페이지 20으로 조정

# 다음 클릭이 이루어질 a 태그의 순서(첫 페이지는 a 태그가 아닌 strong 태그이기 때문에 첫 클릭이 이뤄질 태그는 2부터 시작)
next_a_tag = 2 # 1, 11, 21, ... 첫 페이지는 클릭 불가.

for i in range(1, total_page_num+1):
    if i > THE_LAST_PAGE: break
    print(f'{i}페이지 평점 및 리뷰 정보 수집')
    
    bs = BeautifulSoup(driver.page_source, 'lxml')
    onesReviewBox = bs.find_all('li', attrs={'class':'type01 oneContentTag'}) # 1인 관람평 및 평점 박스
    
    # 첫 페이지 10개 리뷰 저장
    for c in onesReviewBox:
        rating = int(c.find('div', attrs={'class':'story-point'}).text)
        comment = c.find('div', attrs={'class':'story-txt'}).text.strip() # strip으로 리뷰 내 엔터 등의 공백 제거
    #    print(rating, comment) 출력 결과 확인
        ratings.append(rating)
        comments.append(comment)

    if not i%10: # 10번째 페이지 정보 수집이 끝나면(인덱스가 10의 배수이면)
        driver.find_element(By.CSS_SELECTOR, '#contentData > div > div.movie-idv-story > nav > a.control.next').click() # 다음 10페이지 보기 클릭
        time.sleep(PAUSE_TIME)
        next_a_tag = 4 # 다음 10페이지에서 그 다음 클릭이 이뤄질 a 태그의 순서는 4가 됨(다음 클릭할 때 페이지 링크 앞에 <<, <, 첫 페이지가 있기 때문)
        continue
    
    #    다음 페이지 번호 클릭
    driver.find_element(By.CSS_SELECTOR, f'#contentData > div > div.movie-idv-story > nav > a:nth-child({next_a_tag})').click()
    next_a_tag += 1
    time.sleep(PAUSE_TIME)

driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.106)
Stacktrace:
	GetHandleVerifier [0x00BB4CE3+225091]
	(No symbol) [0x00AE4E31]
	(No symbol) [0x00989A7A]
	(No symbol) [0x0096E312]
	(No symbol) [0x009E517B]
	(No symbol) [0x009F55A6]
	(No symbol) [0x009DF2F6]
	(No symbol) [0x009B79B9]
	(No symbol) [0x009B879D]
	sqlite3_dbdata_init [0x01029A83+4064547]
	sqlite3_dbdata_init [0x0103108A+4094762]
	sqlite3_dbdata_init [0x0102B988+4072488]
	sqlite3_dbdata_init [0x00D2C9E9+930953]
	(No symbol) [0x00AF0804]
	(No symbol) [0x00AEAD28]
	(No symbol) [0x00AEAE51]
	(No symbol) [0x00ADCAC0]
	BaseThreadInitThunk [0x75497BA9+25]
	RtlInitializeExceptionChain [0x7787BDAB+107]
	RtlClearBits [0x7787BD2F+191]


In [43]:
print(f'관람객 평점: {sum(ratings)/len(ratings):.1f}점')

import csv # 대용량 데이터 -> 엑셀파일 comma sperate value

with open ('ratings.csv', 'w') as f: # 'w'모드: 텍스트 모드
    writer = csv.writer(f) # csv 파일로 사용가능한 객체 생성
    for rating in ratings:
        writer.writerow([rating]) # 한 행에 쓸 데이터는 리스트 형태로 담아야 함
print('평점 저장 완료')

with open('comment.txt', 'w', encoding='utf-8') as f:
    for comment in comments:
        f.write(comment + '\n')
print('감상평 저장 완료')

ZeroDivisionError: division by zero